# Deep Kernel Learning GP Regression (w/ KISS-GP)

## Overview

In this notebook, we'll give a brief tutorial on how to use deep kernel learning for regression on a medium scale dataset using SKI. This also demonstrates how to incorporate standard PyTorch modules in to a Gaussian process model. 

In [1]:
import math
import torch
import gpytorch
from matplotlib import pyplot as plt
from torch import nn, optim
from gpytorch.kernels import RBFKernel, GridInterpolationKernel, ScaleKernel
from gpytorch.means import ConstantMean
from gpytorch.likelihoods import GaussianLikelihood

# Make plots inline
%matplotlib inline

## Loading Data

For this example notebook, we'll be using the `elevators` UCI dataset used in the paper. Running the next cell downloads a copy of the dataset that has already been scaled and normalized appropriately. For this notebook, we'll simply be splitting the data using the first 80% of the data as training and the last 20% as testing.

**Note**: Running the next cell will attempt to download a ~400 KB dataset file to the current directory.

In [2]:
import urllib.request
import os.path
from scipy.io import loadmat
from math import floor

if not os.path.isfile('elevators.mat'):
    print('Downloading \'elevators\' UCI dataset...')
    urllib.request.urlretrieve('https://drive.google.com/uc?export=download&id=1jhWL3YUHvXIaftia4qeAyDwVxo6j1alk', 'elevators.mat')
    
data = torch.Tensor(loadmat('elevators.mat')['data'])
X = data[:, :-1]
X = X - X.min(0)[0]
X = 2 * (X / X.max(0)[0]) - 1
y = data[:, -1]

# Use the first 80% of the data for training, and the last 20% for testing.
train_n = int(floor(0.8*len(X)))

train_x = X[:train_n, :].contiguous().cuda()
train_y = y[:train_n].contiguous().cuda()

test_x = X[train_n:, :].contiguous().cuda()
test_y = y[train_n:].contiguous().cuda()

## Defining the DKL Feature Extractor

Next, we define the neural network feature extractor used to define the deep kernel. In this case, we use a fully connected network with the architecture `d -> 1000 -> 500 -> 50 -> 2`, as described in the original DKL paper. All of the code below uses standard PyTorch implementations of neural network layers.

In [3]:
data_dim = train_x.size(-1)

class LargeFeatureExtractor(nn.Sequential):           
    def __init__(self):                                      
        super(LargeFeatureExtractor, self).__init__()        
        self.add_module('linear1', nn.Linear(data_dim, 1000))
        self.add_module('relu1', nn.ReLU())                  
        self.add_module('linear2', nn.Linear(1000, 500))     
        self.add_module('relu2', nn.ReLU())                  
        self.add_module('linear3', nn.Linear(500, 50))       
        self.add_module('relu3', nn.ReLU())                  
        self.add_module('linear4', nn.Linear(50, 2))         
                                                             
feature_extractor = LargeFeatureExtractor().cuda()

## Defining the GP Model

We now define the GP model. For more details on the use of GP models, see our simpler examples. This model uses a `GridInterpolationKernel` (SKI) with an RBF base kernel. 

### The forward method

In deep kernel learning, the forward method is where most of the interesting new stuff happens. Before calling the mean and covariance modules on the data as in the simple GP regression setting, we first pass the input data `x` through the neural network feature extractor. Then, to ensure that the output features of the neural network remain in the grid bounds expected by SKI, we scales the resulting features to be between 0 and 1.

Only after this processing do we call the mean and covariance module of the Gaussian process. This example also demonstrates the flexibility of defining GP models that allow for learned transformations of the data (in this case, via a neural network) before calling the mean and covariance function. Because the neural network in this case maps to two final output features, we will have no problem using SKI.

In [4]:
class GPRegressionModel(gpytorch.models.ExactGP):
        def __init__(self, train_x, train_y, likelihood):
            super(GPRegressionModel, self).__init__(train_x, train_y, likelihood)
            self.mean_module = ConstantMean()
            self.base_covar_module = ScaleKernel(RBFKernel())
            self.covar_module = GridInterpolationKernel(self.base_covar_module, num_dims=2, grid_size=100)
            self.feature_extractor = feature_extractor

        def forward(self, x):
            projected_x = self.feature_extractor(x)
            mean_x = self.mean_module(projected_x)
            covar_x = self.covar_module(projected_x)
            return gpytorch.distributions.MultivariateNormal(mean_x, covar_x)

In [5]:
likelihood = GaussianLikelihood().cuda()
model = GPRegressionModel(train_x, train_y, likelihood).cuda()

## Training the model

The cell below trains the DKL model above, learning both the hyperparameters of the Gaussian process **and** the parameters of the neural network in an end-to-end fashion using Type-II MLE. We run 20 iterations of training using the `Adam` optimizer built in to PyTorch. With a decent GPU, this should only take a few seconds.

In [6]:
# Find optimal model hyperparameters
model.train()
likelihood.train()

# Use the adam optimizer
optimizer = torch.optim.SGD([
    {'params': model.feature_extractor.parameters(), 'weight_decay': 1e-3},
    {'params': model.covar_module.parameters()},
    {'params': model.mean_module.parameters()},
    {'params': model.likelihood.parameters()},
], lr=0.1)

# "Loss" for GPs - the marginal log likelihood
mll = gpytorch.mlls.ExactMarginalLogLikelihood(likelihood, model)

training_iterations = 40
def train():
    for i in range(training_iterations):
        # Zero backprop gradients
        optimizer.zero_grad()
        # Get output from model
        output = model(train_x)
        # Calc loss and backprop derivatives
        loss = -mll(output, train_y)
        loss.backward()
        print('Iter %d/%d - Loss: %.3f' % (i + 1, training_iterations, loss.item()))
        optimizer.step()
        
# See dkl_mnist.ipynb for explanation of this flag
with gpytorch.settings.use_toeplitz(False):
    %time train()

Iter 1/40 - Loss: 0.947
Iter 2/40 - Loss: 0.918
Iter 3/40 - Loss: 0.893
Iter 4/40 - Loss: 0.869
Iter 5/40 - Loss: 0.845
Iter 6/40 - Loss: 0.821
Iter 7/40 - Loss: 0.797
Iter 8/40 - Loss: 0.773
Iter 9/40 - Loss: 0.749
Iter 10/40 - Loss: 0.725
Iter 11/40 - Loss: 0.701
Iter 12/40 - Loss: 0.678
Iter 13/40 - Loss: 0.654
Iter 14/40 - Loss: 0.629
Iter 15/40 - Loss: 0.605
Iter 16/40 - Loss: 0.581
Iter 17/40 - Loss: 0.557
Iter 18/40 - Loss: 0.533
Iter 19/40 - Loss: 0.509
Iter 20/40 - Loss: 0.485
Iter 21/40 - Loss: 0.462
Iter 22/40 - Loss: 0.440
Iter 23/40 - Loss: 0.419
Iter 24/40 - Loss: 0.391
Iter 25/40 - Loss: 0.368
Iter 26/40 - Loss: 0.349
Iter 27/40 - Loss: 0.334
Iter 28/40 - Loss: 0.318
Iter 29/40 - Loss: 0.285
Iter 30/40 - Loss: 0.261
Iter 31/40 - Loss: 0.239
Iter 32/40 - Loss: 0.222
Iter 33/40 - Loss: 0.195
Iter 34/40 - Loss: 0.177
Iter 35/40 - Loss: 0.151
Iter 36/40 - Loss: 0.137
Iter 37/40 - Loss: 0.106
Iter 38/40 - Loss: 0.092
Iter 39/40 - Loss: 0.069
Iter 40/40 - Loss: 0.053
CPU times

## Making Predictions

The next cell gets the predictive covariance for the test set (and also technically gets the predictive mean, stored in `preds.mean()`) using the standard SKI testing code, with no acceleration or precomputation. 

In [7]:
model.eval()
likelihood.eval()
with torch.no_grad(), gpytorch.settings.use_toeplitz(False), gpytorch.fast_pred_var():
    preds = model(test_x)

In [8]:
print('Test MAE: {}'.format(torch.mean(torch.abs(preds.mean - test_y))))

Test MAE: 0.08774050325155258
